---
# **DataCoin: Dataset Sazonal para Estratégias Quantitativas com Bitcoin**
---

## **Etapa 1: Importação das Bibliotecas**

In [1]:
# Em caso de erro ao obter dados execute o comando:
# pip install yfinance --upgrade --no-cache-dir
# Ou
# pip uninstall yfinance
# pip install yfinance --upgrade --no-cache-dir

import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

## **Etapa 2: Obtenção dos Dados**

In [2]:
from curl_cffi import requests
session = requests.Session(impersonate="chrome")

inicio = "2017-01-01"
fim = "2025-06-01"

ativo = yf.download("BTC-USD", multi_level_index=False, start=inicio, end=fim, interval="1d", session=session)[["Open", "High", "Low", "Close"]]

C:\Users\Jeovane\AppData\Local\Temp\ipykernel_15832\384073167.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ativo = yf.download("BTC-USD", multi_level_index=False, start=inicio, end=fim, interval="1d", session=session)[["Open", "High", "Low", "Close"]]
[*********************100%***********************]  1 of 1 completed


In [3]:
ativo.head()

,Open,High,Low,Close
Date,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012
2017-01-02,998.617004,1031.390015,996.702026,1021.750000
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005


## **Etapa 3: Preparação dos Dados**

In [4]:
ativo = ativo.resample("ME").agg(
    {"Open": "first", "High": "max", "Low": "min", "Close": "last"}
)

In [5]:
ativo.head()

,Open,High,Low,Close
Date,,,,
2017-01-31,963.658020,1191.099976,755.755981,970.403015
2017-02-28,970.940979,1200.390015,946.690979,1179.969971
2017-03-31,1180.040039,1280.310059,903.713013,1071.790039
2017-04-30,1071.709961,1347.910034,1061.089966,1347.890015
2017-05-31,1348.300049,2763.709961,1348.300049,2286.409912


In [6]:
ativo["Resultado%"] = (ativo["Close"].pct_change() * 100).round(2)

In [7]:
ativo["Mes"] = ativo.index.month
ativo["Ano"] = ativo.index.year
ativo["No.Mes"] = ativo.index.strftime("%m")

In [8]:
ativo = ativo.loc["2004":]

In [9]:
ativo.head()

,Open,High,Low,Close,Resultado%,Mes,Ano,No.Mes
Date,,,,,,,,
2017-01-31,963.658020,1191.099976,755.755981,970.403015,NaN,1,2017,01
2017-02-28,970.940979,1200.390015,946.690979,1179.969971,21.60,2,2017,02
2017-03-31,1180.040039,1280.310059,903.713013,1071.790039,-9.17,3,2017,03
2017-04-30,1071.709961,1347.910034,1061.089966,1347.890015,25.76,4,2017,04
2017-05-31,1348.300049,2763.709961,1348.300049,2286.409912,69.63,5,2017,05


## **Etapa 4: Apresentação dos Dados**

In [ ]:
def getAnaliseMensal(resultado):
    # Cores para cada barra
    cores = ["#5FE8D8" if val > 0 else "#EB4E1C" for val in resultado["Resultado%"]]

    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=resultado.index,
            y=resultado["Resultado%"],
            marker_color=cores,
            text=[f"{v:.2f}" for v in resultado["Resultado%"]],
            textposition="outside",
        )
    )

    # Configurações do layout do gráfico
    fig.update_layout(
        # title="Média Mensal do Resultado (%)",
        title_x=0.5,
        xaxis=dict(
            tickmode="array",
            tickvals=resultado.index,
            ticktext=[
                "Jan", "Fev", "Mar", "Abr", "Mai", "Jun", 
                "Jul", "Ago", "Set", "Out", "Nov", "Dez"
            ],
        ),
        xaxis_title="Mês",
        yaxis_title="Variação Mensal (%)",
        plot_bgcolor="white",
        font=dict(size=14),
        title_font=dict(size=20, family="Arial", color="#222"),
        margin=dict(l=40, r=40, t=60, b=40),
        yaxis=dict(showgrid=True, gridcolor="rgba(0,0,0,0.1)", range=[-5, None]),
    )

    fig.update_traces(marker_line_color="#222", marker_line_width=1.2)
    fig.show()

In [ ]:
def getAnaliseAnualMensal(resultado):
    # Ajusta os meses para ordem correta
    meses = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    mesesNome = [
        "Jan",
        "Fev",
        "Mar",
        "Abr",
        "Mai",
        "Jun",
        "Jul",
        "Ago",
        "Set",
        "Out",
        "Nov",
        "Dez",
    ]

    import numpy as np

    for m in meses:
        if m not in resultado.columns:
            resultado[m] = np.nan
    resultado = resultado[meses]
    resultado.columns = mesesNome

    z = resultado.values
    z_text = np.where(~np.isnan(z), np.vectorize(lambda v: f"{v:.2f}")(z), "")
    colors = np.where(z > 0, "#5FE8D8", "#EB4E1C")
    colors = np.where(np.isnan(z), "#FFFFFF", colors)

    fig = go.Figure(
        data=go.Table(
            header=dict(
                values=["Ano"] + list(resultado.columns),
                fill_color="#222",
                font=dict(color="white", size=18, family="Arial"),
                align="center",
                height=32,
            ),
            cells=dict(
                values=[resultado.index]
                + [resultado[col].values for col in resultado.columns],
                fill_color=[["white"] * len(resultado)]
                + [list(colors[:, i]) for i in range(colors.shape[1])],
                font=dict(color="#222", size=16, family="Arial"),
                format=[None] + [".2f"] * len(resultado.columns),
                align="center",
                height=28,
            ),
        )
    )
    fig.update_layout(
        # title="Resultado Médio (%) por Ano e Mês",
        title_x=0.5,
        margin=dict(l=20, r=20, t=20, b=20),
        font=dict(size=13, family="Arial"),
        plot_bgcolor="white",
        height=325
    )
    fig.show()

In [12]:
resultadoMensal = pd.pivot_table(
    ativo, values="Resultado%", index="No.Mes", columns=[], aggfunc="mean"
)

getAnaliseMensal(resultadoMensal)

In [13]:
resultadoAnualMensal = pd.pivot_table(
    ativo,
    values="Resultado%",
    index=["Ano"],
    columns=["No.Mes"],
    aggfunc="mean",
)

getAnaliseAnualMensal(resultadoAnualMensal)

## **Etapa 5: Salvar Dataset**

In [14]:
ativo.to_csv("DataCoin.csv", index=True, decimal=",", float_format="%.2f")